<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p009_Preprocessing_Layer_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
print(tf.__version__)

2.8.2


# objectif

Jouer avec les layers de preprocessing : 
 * **TASK 1** : simuler données textuelles de training, tokenizer les mots, entrainer RNN dessus, simuler données de test, évaluer le modele
 * **TASK 2** : pareil sauf qu'on tokenize les ngrams
 * **TASK 3** : pareil sauf qu'on tonkenize en pondérant avec TFIDF rien compris

# TASK 1

In [22]:
# Simulation des données
sentences = tf.constant(["The Sky is the limit my man",
                         "I mean, I'm the best bruh",
                         "I like the ragga dance hall so mi go so then",
                         "You know bro, that's the life"])

In [23]:
# Tokenize ce foutu texte bro
vectorizer = tf.keras.layers.TextVectorization(output_mode = "int")
vectorizer.adapt(sentences)

In [24]:
# Exemple
vectorizer(tf.constant(["Touch the sky"])).numpy()

array([[1, 2, 8]])

In [25]:
# Créer un petit modèle avec API fonctionnelle :
# Embedding layer de 16 dimensions
# GRU à 8 units
# Un layer dense qui renvoie du logit

embedding_size = 16
gru_units = 8
input_layer = tf.keras.layers.Input(shape=(None,),dtype="int64")
x = tf.keras.layers.Embedding(input_dim=vectorizer.vocabulary_size(),output_dim=16)(input_layer)
x = tf.keras.layers.GRU(units=8)(x)
output = tf.keras.layers.Dense(units=1,activation="sigmoid")(x)
model = tf.keras.Model(input_layer,output)

In [26]:
# Créer un dataset de phrase de training
# Vectorizer le comme il se doit avec le vectorizer
training_sentences = (["I hate a salad yesterday","My favorite color is purple"],[1,0])
training_dataset = tf.data.Dataset.from_tensor_slices(training_sentences)
training_dataset = training_dataset.map(lambda a,b : (vectorizer(a), b))
training_dataset = training_dataset.batch(2)

In [27]:
# Compiler et trainer le modele
model.compile(loss=tf.keras.losses.BinaryCrossentropy())
model.fit(training_dataset)

1/1 [==============================] - 2s 2s/step - loss: 0.6969


In [28]:
# Pour l'inférence, il faut modifier le modele existant pour qu'il accepte les string as input
input_string = tf.keras.layers.Input(shape=(1,),dtype="string")
x = vectorizer(input_string)
new_output = model(x)
model_inference = tf.keras.Model(input_string,new_output)

In [29]:
# simuler des données de texte et faites l'évaluation du modele dessus
test_sentences = tf.constant(["I always taught that the sky was red","there is no limit about what accomplish in life"])

In [30]:
model_inference(test_sentences)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.5074701],
       [0.5039931]], dtype=float32)>


# TASK 2

In [37]:
# Simuler données
dico_sentences = tf.constant(["Tu sais, j'aime tellement le sex",
                              "J'ai toujours eu envie de te dire que tu étais une réelle merde",
                              "Francisco, il est temps que je t'avoue quelque chose, je suis ton père"])

# Instancier vectorizer ngrams
ngram_vectorizer = tf.keras.layers.TextVectorization(output_mode='multi_hot',ngrams = 2)

# Faire l'adaption
ngram_vectorizer.adapt(dico_sentences)

# Tester avec et sans multi hot
ngram_vectorizer("Tu sais, la vie est belle petite")

<tf.Tensor: shape=(59,), dtype=float32, numpy=
array([1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [38]:
# Designer un petit modele en API fonctionnelle : 
# juste un modele dense apres la couche input multi hot
input_layer = tf.keras.layers.Input(shape=(ngram_vectorizer.vocabulary_size(),),dtype = "float32")
x = tf.keras.layers.Dense(1,activation="sigmoid")(input_layer)
model = tf.keras.Model(input_layer,x)

In [43]:
# Simuler un dataset de training 
training_sentences = ["Il y a ce truc que j'ai jamais osé t'avouer",
 "ce qui compte dans la vie, c'est pas ce que tu fais, c'est ce que tu montres au monde"]
training_dataset = tf.data.Dataset.from_tensor_slices((training_sentences,[1,0]))
training_dataset = training_dataset.map(lambda a,b : (ngram_vectorizer(a),b))
training_dataset = training_dataset.batch(2)

# trainer le modele, compile et fit
model.compile(loss=tf.keras.losses.BinaryCrossentropy())
model.fit(training_dataset)

1/1 [==============================] - 0s 404ms/step - loss: 0.5249


In [46]:
# Simuler des données d'inférence
test_sentences = tf.constant(["Ca me rappelle la fois où ta mère m'a demandé si j'étais homo",
                              "Chousno, voudrais tu faire un tour au lac avec Daddy Yankee?",
                              "Tu penseras à prendre ton sac poubelle avec toi",
                              "Il faudra bien nettoyer les lieux quand on s'en ira"])

# Créer un modèle d'inférence en rajoutant une input layer recevant une string en entrée
input_string = tf.keras.layers.Input(shape=(1,),dtype="string")
x = ngram_vectorizer(input_string)
output = model(x)
model_inference = tf.keras.Model(input_string,output)

# Evaluer ces données
model_inference(test_sentences)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.5442313 ],
       [0.5443399 ],
       [0.53121364],
       [0.5966401 ]], dtype=float32)>


# TASK 3 

In [ ]:
# Pareil que les deux dernieres taches
# Mais utilisez un tfidf multihot
# Modele pareil que la tache 2

In [49]:
# Simuler données
dico_sentences = tf.constant(["Tu sais, j'aime tellement le sex",
                              "J'ai toujours eu envie de te dire que tu étais une réelle merde",
                              "Francisco, il est temps que je t'avoue quelque chose, je suis ton père"])

# Instancier vectorizer ngrams
ngram_vectorizer = tf.keras.layers.TextVectorization(output_mode='tf-idf',ngrams = 2,)

# Faire l'adaption
ngram_vectorizer.adapt(dico_sentences)

# Tester avec et sans multi hot
ngram_vectorizer("Je te dois quelque chose Francisco, je ne suis pas ton père mais ton frère")

<tf.Tensor: shape=(59,), dtype=float32, numpy=
array([15.446135  ,  0.        ,  0.        ,  1.8325815 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.91629076,  1.8325815 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.91629076,
        0.        ,  0.        ,  0.        ,  0.91629076,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.91629076,
        0.91629076,  0.        ,  0.        ,  0.91629076,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.91629076,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.91629076], dtype=float32)>


# A SAVOIR


In [ ]:
# A savoir
# layers.TextVectorization() afin de vectoriser du texte direct sans prise de tête, il se charge de tout le preprocessing
# text_vectorizer.adapt() afin d'initialiser le vectorizer du text
# Pour faire du ngram tokenizer : layers.TextVectorization(output_mode="multi_hot", ngrams=2) # checker pr voir pkoi multi_hot
# Emnedding layer a un input dim la taille du vocabulaire et un output dim la taille de l'espace latent
# Acceder a la taille du vocabulaire d'un vectorizer : vectorizer.vocabulary_size()
# Accéder au vocabulaire d'un vectorizer : vectorizer.get_vocabulary()
# Dans un mapping sur un dataset tensorflow, la fonction lambda doit prendre deux arguments et non un seul quand bien même celui ci serait un tuple
# Multi hot dans un vectorizer de text, c'est juste que ca te renvoie un one hot encoding plutot qu'un integer c 'est tout